# Introduction

Just to understand the images better, and see the capabilities of CNNs on these type of images, and of course playing with the data, I made this simple gender predictor. Taking the images and giving them to 2 layers of convolutions, it is able to predict the gender of above 80% of the images correctly. Here we go:

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pylab as pl
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.layers import Input, InputLayer, Dropout, Conv2D, AveragePooling2D, MaxPooling2D, ReLU, Dense, Flatten, BatchNormalization
from keras.models import Model, Sequential, save_model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras import backend
from sklearn.model_selection import train_test_split


import os
print(os.listdir("../input"))

# Dataset

I have stored the useful part of metadata features in this dataset:

[https://www.kaggle.com/safavieh/siim-acr-pneumothorax-segmentation-metadata](https://www.kaggle.com/safavieh/siim-acr-pneumothorax-segmentation-metadata)

It includes 'ViewPosition', 'PAtientAge', 'PatientSex' for each ImageId.

In [ ]:
trainDF = pd.read_csv('../input/siim-acr-pneumothorax-segmentation-metadata/train.csv')
testDF = pd.read_csv('../input/siim-acr-pneumothorax-segmentation-metadata/test.csv')
trainDF['FileNames'] = trainDF.ImageId.apply(lambda ID: '../input/siim-png-images/input/train_png/' + ID + '.png')
testDF['FileNames'] = testDF.ImageId.apply(lambda ID: '../input/siim-png-images/input/test_png/' + ID + '.png')

# CNN model

Here is the model:

It has two convolutional layer of relatively big size of 16x16 feeding to two layers of dense connections.

In [ ]:
backend.clear_session();

def getModel():
    model = Sequential()
    model.add(InputLayer(input_shape=(128,128,1)))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=16, activation='relu', strides=1))
    model.add(Conv2D(filters=8, kernel_size=16, activation='relu', strides=1))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Flatten())
    model.add(Dropout(.1))
    model.add(BatchNormalization())
    model.add(Dense(20, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer=Adam(lr=0.001, decay=.01),
                  loss='binary_crossentropy',
                  metrics=['binary_accuracy'])
    return model

model = getModel()
model.summary()

Training for 20 epochs:

In [ ]:
trainSet, valSet = train_test_split(trainDF)
params = {'color_mode':'grayscale', 'directory':None, 'x_col':"FileNames", 'y_col':"PatientSex", 'class_mode':"categorical", 'target_size':(128,128)}
trDataGen = ImageDataGenerator().flow_from_dataframe(dataframe=trainSet, batch_size=16, **params)
vlDataGen = ImageDataGenerator().flow_from_dataframe(dataframe=valSet, batch_size=64, shuffle=False, **params)
tsDataGen = ImageDataGenerator().flow_from_dataframe(dataframe=testDF, batch_size=64, shuffle=False, **params)

model.fit_generator(
        trDataGen,
        steps_per_epoch=len(trDataGen)/2,
        epochs=20,
        validation_data=vlDataGen,
        validation_steps=len(vlDataGen))

Results on test set:

In [ ]:
print('LogLoss: %g, binary accuracy: %g' % tuple(model.evaluate_generator(tsDataGen, steps=len(tsDataGen))))

# Model Output

Sample Output:

In [ ]:
batch = tsDataGen[0][0]
Target = tsDataGen[0][1]
Output = model.predict_on_batch(batch)
Labels = ['F','M']
pl.figure(figsize=(15,15))
for i in range(9):
    ax=pl.subplot(3,3,i+1)
    ax.imshow(pl.imread(testDF.FileNames.iloc[i]), cmap='bone')
    ax.set_title('true: '+ testDF.PatientSex.iloc[i] + \
                 ', pred: %.2g%% %s' % (Output[i,:].max()*100, 
                                        Labels[pl.argmax(Output[i,:])]))
    ax.set_axis_off()

# Visulaization of Convolutional Layers

Let's take a look at the trained Convolutional layer weights:

In [ ]:
pl.figure(figsize=(10,10))
w=model.get_layer('conv2d_1').get_weights()[0]
for i in range(w.shape[3]):
    ax=pl.subplot(8,8,i+1)
    ax.imshow(w[:,:,:,i].squeeze())
    ax.set_axis_off()
pl.suptitle('Conv Layer 1')
    
pl.figure(figsize=(10,10))
w=model.get_layer('conv2d_2').get_weights()[0]
for i in range(w.shape[3]):
    ax=pl.subplot(3,3,i+1)
    ax.imshow(w[:,:,:,i].sum(2).squeeze())
    ax.set_axis_off()
pl.suptitle('Conv Layer 2');
